# Sales Prediction

Given historical sales data from from January 2013 to October 2015 of certain items (item_id) in certain shops (shop_id), forecast the sales for November 2015.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

In [2]:
item_categories = pd.read_csv("data/item_categories.csv")
print("item categories")
display(item_categories.head())
items = pd.read_csv("data/items.csv")
print("items ")
display(items.head())
sales_train = pd.read_csv("data/sales_train.csv")
print("train set")
display(sales_train.head())
shops = pd.read_csv("data/shops.csv")
print("shops ")
display(shops.head())

# for later
print("test data")
test = pd.read_csv("data/test.csv")
display(test.head())
sample_submission = pd.read_csv("data/sample_submission.csv")
print("submission test")
display(sample_submission.head())

item categories


,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


items 


,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


train set


...

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


test data


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


submission test


,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


### Save some memory adjusting types

In [3]:
display(sales_train.shape)
display(sales_train.describe())
display(sales_train.dtypes)
initial_mem_usage = sales_train.memory_usage(deep = True).sum()
sales_train =  sales_train.assign(
        date = pd.to_datetime(sales_train.date, format='%d.%m.%Y'),
        date_block_num = sales_train.date_block_num.astype('int8'),
        shop_id = sales_train.shop_id.astype('int8')
    )
print(f"saved {(initial_mem_usage - sales_train.memory_usage(deep=True).sum())/initial_mem_usage*100:.2f}%")

(2935849, 6)

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


date               object
date_block_num      int64
shop_id             int64
item_id             int64
item_price        float64
item_cnt_day      float64
dtype: object

saved 68.22%


In [4]:
items.describe()
items = items.assign(item_category_id = items.item_category_id.astype('int8'))

### Join item categories with train set

In [26]:
sales_train['date'].dt.is_month_end

0          Wednesday
1           Thursday
2           Saturday
3             Sunday
4            Tuesday
             ...    
2935844     Saturday
2935845       Friday
2935846    Wednesday
2935847     Thursday
2935848     Saturday
Name: date, Length: 2935849, dtype: object

In [27]:
features = ['date', 'shop_id', 'item_id', 'item_price', 'item_cnt_day']
train_joined = (sales_train[features]
                .join(items[['item_id', 'item_category_id']],
                       on='item_id', how = 'inner', rsuffix='_other')
                .drop(columns='item_id_other')
            .assign(month = lambda x: x['date'].dt.month,
                    day_of_year = lambda x:x['date'].dt.day_of_year,
                    day_of_week = lambda x: x['date'].dt.day_of_week,
                    is_month_end = lambda x:x['date'].dt.is_month_end,
                    is_month_start = lambda x:x['date'].dt.is_month_start,
                    is_year_start = lambda x:x['date'].dt.is_year_start,
                    is_year_end = lambda x:x['date'].dt.is_year_end
                    )
            .set_index('date'))
display(train_joined)

,shop_id,item_id,item_price,item_cnt_day,item_category_id,month,day_of_year,day_of_week,is_month_end,is_month_start,is_year_start,is_year_end
date,,,,,,,,,,,,
2013-01-02,59,22154,999.0,1.0,37,1,2,2,False,False,False,False
2013-01-23,24,22154,999.0,1.0,37,1,23,2,False,False,False,False
2013-01-20,27,22154,999.0,1.0,37,1,20,6,False,False,False,False
2013-01-02,25,22154,999.0,1.0,37,1,2,2,False,False,False,False
2013-01-03,25,22154,999.0,1.0,37,1,3,3,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
2015-10-17,25,8428,249.0,1.0,40,10,290,5,False,False,False,False
2015-10-01,25,7903,12198.0,1.0,15,10,274,3,False,True,False,False
2015-10-29,25,7610,2890.0,1.0,64,10,302,3,False,False,False,False


In [19]:
print(f"number of different shops {train_joined['shop_id'].nunique()}")
print(f"number of different items {train_joined['item_id'].nunique()}")
print(f"different item categories {train_joined['item_category_id'].nunique()}")

number of different shops 60
number of different items 21807
different item categories 84


In [17]:
# distribution of observation per item
train_joined['item_id'].value_counts().describe()

count    21807.000000
mean       134.628743
std        406.938186
min          1.000000
25%          6.000000
50%         32.000000
75%        119.000000
max      31340.000000
Name: count, dtype: float64